# Code này được setup để chạy trên Colab hoặc các môi trường có cuda. Nếu không có thì sửa thành device cpi

In [1]:
!gdown 1XZpEYxkPkDLU8U863ZBj1_7Qg7tLAybZ
!unzip Private_test_traffic

Downloading...
From (original): https://drive.google.com/uc?id=1XZpEYxkPkDLU8U863ZBj1_7Qg7tLAybZ
From (redirected): https://drive.google.com/uc?id=1XZpEYxkPkDLU8U863ZBj1_7Qg7tLAybZ&confirm=t&uuid=cfd78978-b577-47f0-9cb0-9ba588415427
To: /content/Private_test_traffic.zip
100% 156M/156M [00:01<00:00, 103MB/s]
Archive:  Private_test_traffic.zip
   creating: private test/
  inflating: private test/cam_11_00001.jpg  
  inflating: private test/cam_11_00002.jpg  
  inflating: private test/cam_11_00003.jpg  
  inflating: private test/cam_11_00004.jpg  
  inflating: private test/cam_11_00005.jpg  
  inflating: private test/cam_11_00006.jpg  
  inflating: private test/cam_11_00007.jpg  
  inflating: private test/cam_11_00008.jpg  
  inflating: private test/cam_11_00009.jpg  
  inflating: private test/cam_11_00010.jpg  
  inflating: private test/cam_11_00011.jpg  
  inflating: private test/cam_11_00012.jpg  
  inflating: private test/cam_11_00013.jpg  
  inflating: private test/cam_11_00014.jpg  

In [2]:
!git clone https://github.com/Peterande/D-FINE.git
%cd D-FINE
!pip install -r requirements.txt

Cloning into 'D-FINE'...
remote: Enumerating objects: 899, done.
remote: Counting objects: 100% (608/608), done.
remote: Compressing objects: 100% (307/307), done.
remote: Total 899 (delta 359), reused 489 (delta 293), pack-reused 291 (from 1)
Receiving objects: 100% (899/899), 388.81 KiB | 9.04 MiB/s, done.
Resolving deltas: 100% (485/485), done.
/content/D-FINE
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.9/470.9 kB 15.6 MB/s eta 0:00:00


In [3]:
%%writefile ./configs/dataset/custom_detection.yml

task: detection

evaluator:
  type: CocoEvaluator
  iou_types: ['bbox']

num_classes: 4  # Thay bằng số lớp của bạn

remap_mscoco_category: False

train_dataloader:
  type: DataLoader
  dataset:
    type: CocoDetection
    img_folder: /content/coco_data/images/train
    ann_file: /content/coco_data/annotations/instances_train.json
    return_masks: False
    transforms:
      type: Compose
      ops: ~
  shuffle: True
  num_workers: 2
  drop_last: True
  collate_fn:
    type: BatchImageCollateFunction

val_dataloader:
  type: DataLoader
  dataset:
    type: CocoDetection
    img_folder: /content/coco_data/images/val
    ann_file: /content/coco_data/annotations/instances_val.json
    return_masks: False
    transforms:
      type: Compose
      ops: ~
  shuffle: False
  num_workers: 2
  drop_last: False
  collate_fn:
    type: BatchImageCollateFunction

Overwriting ./configs/dataset/custom_detection.yml


In [4]:
%%writefile ./configs/dfine/custom/objects365/dfine_hgnetv2_s_obj2custom.yml

__include__: [
  '../../../dataset/custom_detection.yml',
  '../../../runtime.yml',
  '../../include/dataloader.yml',
  '../../include/optimizer.yml',
  '../../include/dfine_hgnetv2.yml',
]

output_dir: ./output/dfine_hgnetv2_s_custom


DFINE:
  backbone: HGNetv2

HGNetv2:
  name: 'B0'
  return_idx: [1, 2, 3]
  freeze_at: -1
  freeze_norm: False
  use_lab: True

DFINETransformer:
  num_layers: 3  # 4 5 6
  eval_idx: -1  # -2 -3 -4

HybridEncoder:
  in_channels: [256, 512, 1024]
  hidden_dim: 256
  depth_mult: 0.34
  expansion: 0.5

optimizer:
  type: AdamW
  params:
    -
      params: '^(?=.*backbone)(?!.*norm|bn).*$'
      lr: 0.0002
    -
      params: '^(?=.*backbone)(?=.*norm|bn).*$'
      lr: 0.0002
      weight_decay: 0.
    -
      params: '^(?=.*(?:encoder|decoder))(?=.*(?:norm|bn|bias)).*$'
      weight_decay: 0.

  lr: 0.0004
  betas: [0.9, 0.999]
  weight_decay: 0.0001


# Increase to search for the optimal ema
epoches: 220
train_dataloader:
  total_batch_size: 4
  dataset:
    transforms:
      policy:
        epoch: 200
  collate_fn:
    stop_epoch: 200
    ema_restart_decay: 0.9999
    base_size_repeat: 20

val_dataloader:
  total_batch_size: 8

Overwriting ./configs/dfine/custom/objects365/dfine_hgnetv2_s_obj2custom.yml


In [5]:
!pip install -r tools/inference/requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.6.0.post1-py2.py3-none-any.whl size=16416 sha256=e5d5a34322f2f18257822947f4ad9ee1741adc12d82bfc3bff96f4261079f4df
  Stored in directory: /root/.cache/pip/wheels/d6/d4/72/a641e4695bb1b8b57fa2ad1235493c73c469a080b16e5375a9
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.6.0.post1-py2.py3-none-any.whl size=17631 sha256=7da7306b785ca23c3b57c4da53ea0c8b022d692923b57cb81215584033dc8494
  Stored in directory: /root/.cache/pip/wheels/aa/cb/33/33a230ed5c2a54da56fd68d4edd3adff93b82812124fbf9efd
Successfully built tensorrt tensorrt-cu12


In [6]:
!gdown 1wT3EuyXdJoBlLd61BFGBTM6N3lUpwHZJ
!unzip best_stg1.pth

Downloading...
From (original): https://drive.google.com/uc?id=1wT3EuyXdJoBlLd61BFGBTM6N3lUpwHZJ
From (redirected): https://drive.google.com/uc?id=1wT3EuyXdJoBlLd61BFGBTM6N3lUpwHZJ&confirm=t&uuid=cfb1642f-730c-4eac-9739-ab967c9a1226
To: /content/D-FINE/best_stg1.pth.zip
100% 152M/152M [00:02<00:00, 65.1MB/s]
Archive:  best_stg1.pth.zip
  inflating: best_stg1.pth           


In [ ]:
# import requests

# def download_image(url, save_path):
#     try:
#         response = requests.get(url, stream=True)
#         response.raise_for_status()  # Kiểm tra nếu có lỗi HTTP

#         with open(save_path, 'wb') as file:
#             for chunk in response.iter_content(1024):
#                 file.write(chunk)

#         print(f"Ảnh đã được tải xuống và lưu tại: {save_path}")
#     except requests.exceptions.RequestException as e:
#         print(f"Lỗi khi tải ảnh: {e}")

# # Ví dụ
# image_url = "https://images.pexels.com/photos/2346915/pexels-photo-2346915.jpeg?cs=srgb&dl=pexels-arthurbrognoli-2346915.jpg&fm=jpg"  # URL ảnh
# save_path = "image.jpg"          # Đường dẫn lưu
# download_image(image_url, save_path)

Ảnh đã được tải xuống và lưu tại: image.jpg


In [8]:
%%writefile ./tools/inference/bkai_inf.py

"""
Copyright (c) 2024 The D-FINE Authors. All Rights Reserved.
"""

import torch
import torch.nn as nn
import torchvision.transforms as T

import numpy as np
from PIL import Image

import sys
import os
from tqdm import tqdm

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '../../')))
from src.core import YAMLConfig


def process_folder(model, device, folder_path, output_file):
    image_files = [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if os.path.splitext(f)[-1].lower() in ['.jpg', '.jpeg', '.png', '.bmp']
    ]
    image_files = sorted(image_files)
    with open(output_file, 'w') as f:
        for img_file in tqdm(image_files, desc="Processing images"):
            im_pil = Image.open(img_file).convert('RGB')
            w, h = im_pil.size
            orig_size = torch.tensor([[w, h]]).to(device)

            transforms = T.Compose([
                T.Resize((640, 640)),
                T.ToTensor(),
            ])
            im_data = transforms(im_pil).unsqueeze(0).to(device)

            output = model(im_data, orig_size)
            labels, boxes, scores = output

            # Apply confidence and area thresholds
            CONF_THRESHOLD = 0.45
            MAX_AREA_THRESHOLD_RATIO = 0.3
            scr = scores[0]
            lab = labels[0]
            box = boxes[0]
            scrs = scr

            keep = scrs >= CONF_THRESHOLD
            scrs = scrs[keep]
            lab = lab[keep]
            box = box[keep]

            for j in range(len(box)):
                box_j = box[j].detach().cpu().numpy()
                x_min, y_min, x_max, y_max = box_j
                class_id = int(lab[j].item())
                confidence = scrs[j].item()
                area = (x_max - x_min) * (y_max - y_min)
                max_area_threshold = MAX_AREA_THRESHOLD_RATIO * w * h
                if area <= max_area_threshold:
                    # Convert to normalized xywh
                    x_center = ((x_min + x_max) / 2) / w
                    y_center = ((y_min + y_max) / 2) / h
                    width = (x_max - x_min) / w
                    height = (y_max - y_min) / h

                    # Write to file
                    img_name = os.path.basename(img_file)
                    f.write(f"{img_name} {class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {confidence:.6f}\n")


def main(args):
    """Main function"""
    cfg = YAMLConfig(args.config, resume=args.resume)

    if 'HGNetv2' in cfg.yaml_cfg:
        cfg.yaml_cfg['HGNetv2']['pretrained'] = False

    if args.resume:
        checkpoint = torch.load(args.resume, map_location='cpu')
        if 'ema' in checkpoint:
            state = checkpoint['ema']['module']
        else:
            state = checkpoint['model']
    else:
        raise AttributeError('Only support resume to load model.state_dict by now.')

    # Load train mode state and convert to deploy mode
    cfg.model.load_state_dict(state)

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.model = cfg.model.deploy()
            self.postprocessor = cfg.postprocessor.deploy()

        def forward(self, images, orig_target_sizes):
            outputs = self.model(images)
            outputs = self.postprocessor(outputs, orig_target_sizes)
            return outputs

    device = args.device
    model = Model().to(device)
    model.eval()

    # Check if the input is a folder
    file_path = args.input
    if os.path.isdir(file_path):
        # Process as a folder
        output_file = "predict.txt"
        process_folder(model, device, file_path, output_file)
        print("Processing complete. Results saved in 'predict.txt'.")
    else:
        print("Please provide a folder containing images.")
        sys.exit(1)


if __name__ == '__main__':
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('-c', '--config', type=str, required=True)
    parser.add_argument('-r', '--resume', type=str, required=True)
    parser.add_argument('-i', '--input', type=str, required=True)
    parser.add_argument('-d', '--device', type=str, default='cpu')
    args = parser.parse_args()
    main(args)

Writing ./tools/inference/bkai_inf.py


In [10]:
!python tools/inference/bkai_inf.py \
  -c ./configs/dfine/custom/objects365/dfine_hgnetv2_s_obj2custom.yml \
  -r best_stg1.pth \
  --input /content/private\ test/ \
  --device cuda

2024-12-02 15:39:26.889953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 15:39:26.910778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 15:39:26.916851: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 15:39:26.930616: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-02 15:39:28.218627: W tensorflow/compiler/tf2

In [12]:
!zip -r submission.zip predict.txt

  adding: predict.txt (deflated 71%)
